#### Report Usage Facts

##### Data ingestion strategy:
<mark style="background: #88D5FF;">**MERGE**</mark>

##### Related pipeline:

**Ext_Load_PBI_Report_Usage_E2E**

##### Source:

**Files** from FUAM_Ext_Lakehouse folder **bronze_file_location** variable

##### Target:

**1 Delta table** in FUAM_Ext_Lakehouse 
- **gold_table_name** variable value


In [ ]:
## Parameters
display_data = True

usage_table_name = "Report views"
bronze_file_location = "Files/raw/report_usage/facts/report_views/"
gold_table_name = "report_views"
usage_dataset_id = "a4c91678-f53c-479b-b3d2-bf573ef36660"

print("Successfully configured all paramaters for this run.")

In [ ]:
## Import all packages used in this notebook
import datetime
from delta.tables import DeltaTable
from pyspark.sql.functions import col, explode, to_date, date_format, lit, upper, row_number # type: ignore
from pyspark.sql.window import Window # type ignore
from pyspark.sql import SparkSession # type: ignore
import random
import re
import time

print("Successfully imported all packages for this notebook.")

In [ ]:
#
# Create the Spark session
#
app_name = "TransferReportFacts"

# Get the current Spark session
spark = SparkSession.builder \
    .appName(app_name) \
    .getOrCreate()

print(f"Spark session {app_name} has been created successfully.")

In [ ]:
#
# Function to get check if the gold layer table already exists
#
def gold_table_exists(gold_table_name: str, spark) -> bool:
    """
    Checks if a table exists in the FUAM_Ext_Lakehouse catalog.

    Args:
        gold_table_name (str): Name of the table to check.
        spark (SparkSession): The active Spark session.

    Returns:
        bool: True if the table exists, False otherwise.
    """
    table_exists = spark._jsparkSession.catalog().tableExists('FUAM_Ext_Lakehouse', gold_table_name)
    return table_exists

print("The function 'gold_table_exists' has been created successfully.") 

In [ ]:
#
# Function to get the refresh date for UPSERT processing from the SILVER and GOLD layers
#
def get_refresh_date(gold_table_name: str, dataset_id: str, silver_df, spark) -> datetime:
    spark = silver_df.sparkSession  # Get SparkSession from the DataFrame

    # Get earlist date from silver_df
    silver_min_df = silver_df.select(col('CreationDate')).orderBy(col('CreationDate'), ascending=True).first()
    silver_earlist_date = silver_min_df['CreationDate'] if silver_min_df else None

    if gold_table_exists(gold_table_name, spark):
        # Get latest date from gold table
        get_latest_date_sql = f"""
            SELECT CreationDate 
            FROM FUAM_Ext_Lakehouse.{gold_table_name}
            WHERE UsageDatasetId = '{dataset_id}'
            ORDER BY CreationDate DESC 
            LIMIT 1
        """
        gold_max_df = spark.sql(get_latest_date_sql)

        if gold_max_df.count() == 0:
            print("No existing records in gold table. Using date from silver_df.")
            refresh_date = silver_earlist_date
        else:
            print("Using date from gold.")
            refresh_date = gold_max_df.first()['CreationDate']
    else:
        print("Using date from silver_df.")
        refresh_date = silver_earlist_date

    print(f"Refresh start date: {refresh_date}")
    return refresh_date

print("The function 'get_refresh_date' has been created successfully.") 

In [ ]:
#
# Get the report usage data from the BRONZE layer
# Read the JSON files using 'multiline' since it's pretty-printed
#
raw_location = f"{bronze_file_location}{usage_dataset_id.upper()}.json"
bronze_df = spark.read.option("multiline", "true").json(raw_location)

print(f"Bronze data from {raw_location} has been read successfully.")

In [ ]:
if display_data:
    display(bronze_df)

In [ ]:
#
# Explode results -> tables -> rows
#
exploded_results = bronze_df.select(explode("results").alias("result"))
exploded_tables = exploded_results.select(explode("result.tables").alias("table"))
exploded_rows = exploded_tables.select(explode("table.rows").alias("row"))

print(f"Bronze data from {raw_location} has been extracted and transformed.")

In [ ]:
if display_data:
    display(exploded_rows)

In [ ]:
#
# Create the silver layer dataframe
# Dataset IDs in the Lakehouse are expected to be uppercase
#
usage_dataset_id = usage_dataset_id.upper()

# Create the silver dataframe
silver_df = exploded_rows.select(
    lit(usage_dataset_id).alias("UsageDatasetId"), # Add to enable the append processing logic
    col("row.*")
)

# Rename columns with brackets to just the inner name
for col_name in silver_df.columns:
    match = re.search(r"\[(.*?)\]", col_name)
    if match:
        new_col_name = match.group(1)
        silver_df = silver_df.withColumnRenamed(col_name, new_col_name)

# Put selected ID columns to Upper Case
for co in silver_df.columns:
    if co in ['CapacityId','WorkspaceId', 'ReportId']:
        silver_df = silver_df.withColumn(co, upper(silver_df[co]))

# Transformation to standardize the time column
if "Timestamp" in silver_df.columns and "CreationTime" not in silver_df.columns:
    silver_df = silver_df.withColumnRenamed("Timestamp", "CreationTime")

# Transform the time stamp string data field
silver_df = silver_df.withColumn("CreationDate", to_date(col("CreationTime").substr(1, 10), "yyyy-MM-dd")) \
                    .withColumn("CreationDateKey", date_format(col("CreationTime"), "yyyyMMdd")) \
                    .withColumn("CreationHour", date_format(col("CreationTime"), "H")) \
                    .withColumn("CreationMinute", date_format(col("CreationTime"), "mm"))

print(f"Silver dataframe has been created successfully with {silver_df.count()} rows.")

In [ ]:
#
# Transform the silver layer dataframe
# Get the date to start append processing
#
refresh_date = get_refresh_date(gold_table_name, usage_dataset_id, silver_df, spark) 
    
# Filter silver_df data based on reresh date
silver_df = silver_df.filter(col("CreationDate") >= lit(refresh_date))

print(f"Silver dataframe has been filtered successfully on/after {refresh_date} with {silver_df.count()} rows.")

In [ ]:
if display_data:
    display(silver_df)

In [ ]:
#
# Configure the MERGE process
#
max_retries = 5
retry_delay = 60  # max delay in seconds
success = False  # Flag to indicate success

if gold_table_name == 'report_views':
    match_criteria = f"""
        target.ReportId = source.ReportId AND
        target.CreationTime = source.CreationTime AND
        target.UserId = source.UserId AND
        target.OriginalConsumptionMethod = source.OriginalConsumptionMethod
    """
else:
    match_criteria = f"""
        target.ReportId = source.ReportId AND
        target.CreationTime = source.CreationTime AND
        target.UserId = source.UserId AND
        target.Client = source.Client
    """

# Replace all non-alphanumeric characters (except spaces) with nothing
match_criteria = match_criteria.replace('\n', ' ').replace('\r', ' ')
# Replace multiple spaces with a single space
match_criteria = re.sub(r'\s+', ' ', match_criteria)
# Trim leading/trailing whitespace
match_criteria = match_criteria.strip()

print("Successfully configured the merge process.")

In [ ]:
#
# De-dupe the the SILVER layer dataframe to avoid MERGE match issues
#
if gold_table_name == 'report_views':
    window_spec = Window.partitionBy("ReportId", "CreationTime", "UserId", "OriginalConsumptionMethod").orderBy("CreationTime")
else:
    window_spec = Window.partitionBy("ReportId", "CreationTime", "UserId", "Client").orderBy("CreationTime")


# Keep only the first row for each group of duplicates
deduped_df = silver_df.withColumn("row_num", row_number().over(window_spec)).filter("row_num = 1").drop("row_num")

print(f"{gold_table_name} before deduping contains {silver_df.count()} rows.")
print(f"{gold_table_name} after deduping contains {deduped_df.count()} rows.")

In [ ]:
#
# Perform the merge to insert new and update existing records (UPSERT approach)
#

# Load the target Delta table
delta_table = DeltaTable.forName(spark, gold_table_name)

# Perform the MERGE operation with auto-update and insert
for attempt in range(max_retries):
    try:
        delta_table.alias("target") \
            .merge(deduped_df.alias("source"), match_criteria) \
            .whenMatchedUpdateAll() \
            .whenNotMatchedInsertAll() \
            .execute()
        success = True
        break  # Exit loop if successful
    except Exception as e:
        if "ConcurrentAppendException" in str(e):
            wait_time = random.randint(1, retry_delay)
            print(f"Retrying due to concurrent append conflict... Attempt {attempt + 1}, sleeping {wait_time} seconds")
            time.sleep(wait_time)
        else:
            raise e  # Raise other errors

if not success:
    print(f"Merge operation for gold table {gold_table_name} failed after {max_retries} retries.")
else:
    print(f"Gold table {gold_table_name} has been merged successfully.")

In [ ]:
#
# Write history of bronze files
#
raw_path = bronze_file_location.replace("*/", '', )
history_path = raw_path.replace("Files/raw/", "Files/history/")
mssparkutils.fs.cp(raw_path, history_path + datetime.datetime.now().strftime('%Y/%m/%d') + "/", True) # type: ignore

print(f"History data copied to {history_path} successfully.")